In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
from pathlib import Path
from urllib.parse import urlparse, unquote
import requests
from tqdm import tqdm
import fitz  
import zipfile

In [48]:
def safe_filename(name: str) -> str:
    """Limpia nombres raros para guardar en disco."""
    name = unquote(name)
    name = re.sub(r"[^\w\-.() ]+", "_", name, flags=re.UNICODE)
    name = re.sub(r"\s+", " ", name).strip()
    return name


def filename_from_url(url: str) -> str:
    p = urlparse(url)
    base = os.path.basename(p.path) or "archivo"
    base = safe_filename(base)
    return base


def download_file(url: str, out_path: Path, timeout=60):
    """Descarga con streaming y reintentos simples."""
    out_path.parent.mkdir(parents=True, exist_ok=True)

    # Si ya existe y pesa algo, no lo bajes de nuevo
    if out_path.exists() and out_path.stat().st_size > 0:
        return "skip"

    headers = {"User-Agent": "Mozilla/5.0 (compatible; bulk-downloader/1.0)"}

    with requests.get(url, stream=True, timeout=timeout, headers=headers) as r:
        r.raise_for_status()
        total = int(r.headers.get("Content-Length", 0))
        tmp = out_path.with_suffix(out_path.suffix + ".part")

        with open(tmp, "wb") as f, tqdm(
            total=total if total > 0 else None,
            unit="B",
            unit_scale=True,
            desc=out_path.name,
            leave=False,
        ) as pbar:
            for chunk in r.iter_content(chunk_size=1024 * 256):
                if chunk:
                    f.write(chunk)
                    if total > 0:
                        pbar.update(len(chunk))

        tmp.replace(out_path)

    return "ok"


def extract_links_from_pdf(pdf_path: Path):
    """Extrae URIs de enlaces embebidos en el PDF."""
    doc = fitz.open(pdf_path)
    links = []
    for page in doc:
        for link in page.get_links():
            uri = link.get("uri")
            if uri:
                links.append(uri)
    doc.close()
    # quitar duplicados preservando orden
    seen = set()
    uniq = []
    for u in links:
        if u not in seen:
            seen.add(u)
            uniq.append(u)
    return uniq


def main(index_pdf_url: str, out_dir="sarampion_2025"):
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    index_pdf_path = out_dir / "indice.pdf"
    print("Descargando índice PDF…")
    download_file(index_pdf_url, index_pdf_path)

    print("Extrayendo links del PDF…")
    links = extract_links_from_pdf(index_pdf_path)
    print(f"Links encontrados: {len(links)}")

    wanted_ext = (".zip", ".csv", ".xlsx", ".xls", ".txt", ".pdf")
    links = [u for u in links if urlparse(u).path.lower().endswith(wanted_ext)]
    print(f"Links tras filtro por extensión: {len(links)}")

    ok = skip = fail = 0
    print("Descargando archivos…")
    for url in tqdm(links, desc="Total", unit="archivo"):
        fname = filename_from_url(url)
        out_path = out_dir / fname
        try:
            status = download_file(url, out_path)
            if status == "ok":
                ok += 1
            elif status == "skip":
                skip += 1
        except Exception as e:
            fail += 1
            print(f"\n[ERROR] {url}\n  -> {e}\n")

    print("\nResumen:")
    print("  descargados:", ok)
    print("  ya existían:", skip)
    print("  fallidos:", fail)
    print("Carpeta:", out_dir.resolve())


if __name__ == "__main__":

    INDEX_PDF_URL = "https://www.gob.mx/cms/uploads/attachment/file/1041077/datos_abiertos_historicos_efe_2025.pdf"
    main(INDEX_PDF_URL, out_dir="datos_semanales_2025")


Descargando índice PDF…
Extrayendo links del PDF…
Links encontrados: 43
Links tras filtro por extensión: 43
Descargando archivos…


Total: 100%|███████████████████████████████| 43/43 [00:00<00:00, 83.96archivo/s]


[ERROR] https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/efe/historicos/2025/datos_abiertos_efe_0050225.zip
  -> 404 Client Error: Not Found for url: https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/efe/historicos/2025/datos_abiertos_efe_0050225.zip


Resumen:
  descargados: 0
  ya existían: 42
  fallidos: 1
Carpeta: /Users/juanalbertomartinez/Desktop/Datos_Sarampión/datos_semanales_2025


Semanas faltantes:

2023  -> 404 Client Error: Not Found for url: https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/efe/historicos/2023/datos_abiertos_efe_150523.zip

2024  -> 404 Client Error: Not Found for url: https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/efe/historicos/2024/datos_abiertos_efe_280624.zip

2024  -> 404 Client Error: Not Found for url: https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/efe/historicos/2024/datos_abiertos_efe_040124.zip

2025 -> 404 Client Error: Not Found for url: https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/efe/historicos/2025/datos_abiertos_efe_0050225.zip


In [49]:
ruta_base = "." 
todos_los_dfs = []  

for carpeta in sorted(os.listdir(ruta_base)):

    if carpeta.startswith("datos_semanales") and os.path.isdir(carpeta):
        ruta_carpeta = os.path.join(ruta_base, carpeta)
        print(f"--> Procesando carpeta: {carpeta}")
        
        for archivo in os.listdir(ruta_carpeta):
            if archivo.endswith(".zip"):
                ruta_zip = os.path.join(ruta_carpeta, archivo)
                
                try:

                    with zipfile.ZipFile(ruta_zip, "r") as z:
 
                        for nombre_archivo_interno in z.namelist():
                            if nombre_archivo_interno.endswith(".csv"):
                                with z.open(nombre_archivo_interno) as f:
                        
                                    df_temporal = pd.read_csv(f, encoding='utf-8') 
                                    todos_los_dfs.append(df_temporal)
                                break 
                except Exception as e:
                    print(f"Error leyendo {archivo}: {e}")


if todos_los_dfs:
    df_final = pd.concat(todos_los_dfs, ignore_index=True)
    

    df_final.to_csv("datos_concatenados_final.csv", index=False)
    print(f"Se han unido {len(todos_los_dfs)} archivos.")
    print(f"Dimensiones finales: {df_final.shape}")
else:
    print("No se encontraron archivos CSV para unir.")

--> Procesando carpeta: datos_semanales_2020
--> Procesando carpeta: datos_semanales_2021
--> Procesando carpeta: datos_semanales_2022
--> Procesando carpeta: datos_semanales_2023
--> Procesando carpeta: datos_semanales_2024
--> Procesando carpeta: datos_semanales_2025
--> Procesando carpeta: datos_semanales_2026
Se han unido 232 archivos.
Dimensiones finales: (474583, 22)


In [50]:
# Verificación de duplicados por nombre (Mayúsculas vs Minúsculas)
cols_originales = df_final.columns
cols_minusculas = [c.lower().strip() for c in cols_originales]

if len(set(cols_minusculas)) != len(cols_originales):
    print("Hay columnas que parecen ser la misma pero están escritas diferente.")
    
    # Identificar cuáles son
    import collections
    duplicados = [item for item, count in collections.Counter(cols_minusculas).items() if count > 1]
    print(f"Posibles columnas repetidas: {duplicados}")
else:
    print("No hay conflictos de mayúsculas/minúsculas en los nombres de columnas.")

print("-" * 30)

# Valores Nulos 

nulos = df_final.isnull().sum()
nulos_filtrados = nulos[nulos > 0] 

if not nulos_filtrados.empty:
    print("\nColumnas con valores vacíos (cantidad):")
    print(nulos_filtrados.sort_values(ascending=False))
else:
    print("\n  El dataset está completo, no hay valores nulos.")

print("-" * 30)


print(f"\nTotal de columnas detectadas: {len(df_final.columns)}")
print(df_final.columns.tolist())

No hay conflictos de mayúsculas/minúsculas en los nombres de columnas.
------------------------------

Columnas con valores vacíos (cantidad):
ORIGEN_CASO          11330
FIEBRE                   6
FECHA_DIAGNOSTICO        1
dtype: int64
------------------------------

Total de columnas detectadas: 22
['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'EDAD_ANOS', 'EDAD_MESES', 'EDAD_DIAS', 'SEXO', 'HABLA_LENGUA_INDIG', 'INDIGENA', 'ENTIDAD_UM_NOTIF', 'MUNICIPIO_UM_NOTIF', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'INSTITUCION_NOTIF', 'VACUNACION', 'EXANTEMA', 'FIEBRE', 'COMPLICACIONES', 'DEFUNCION', 'DIAGNOSTICO', 'CRITERIO_DIAGNOSTICO', 'FECHA_DIAGNOSTICO', 'ORIGEN_CASO']


In [51]:
df = pd.read_csv("datos_concatenados_final.csv")
df_confirmados = df[df['DIAGNOSTICO'] == 1].copy()

print(f"Registros originales: {len(df)}")
print(f"Registros filtrados (Diagnóstico == 1): {len(df_confirmados)}")
print(f"Porcentaje conservado: {(len(df_confirmados)/len(df)*100):.2f}%")

Registros originales: 474583
Registros filtrados (Diagnóstico == 1): 109458
Porcentaje conservado: 23.06%


Para el análisis de la evolución temporal de los casos confirmados de sarampión, se ha optado por utilizar la variable FECHA_ACTUALIZACION en lugar de FECHA_DIAGNOSTICO por las siguientes razones técnicas:

- Integridad de los Datos: La variable FECHA_DIAGNOSTICO presenta un alto índice de valores no informados (codificados como 9999-99-99), lo que representa una pérdida de aproximadamente el 87% de los registros. El uso exclusivo de esta variable sesgaría significativamente el estudio al descartar la gran mayoría de la muestra.

- Continuidad Operativa: La columna FECHA_ACTUALIZACION cuenta con un 100% de completitud (non-null). Representa el momento en que el registro fue validado o modificado en el sistema oficial, funcionando como un proxy (indicador indirecto) confiable para el seguimiento de la vigilancia epidemiológica.

- Análisis de Tendencias: Dado que el objetivo es identificar brotes y picos de contagio, la fecha de actualización permite agrupar los casos en una línea de tiempo continua sin interrupciones, manteniendo la representatividad de los 109,458 casos confirmados.

- Nota: Se asume un desfase administrativo (tiempo de notificación) entre el diagnóstico real y la actualización, sin embargo, para fines de tendencia anual y mensual, esta métrica es estadísticamente más robusta que trabajar con una muestra fragmentada.

In [52]:
df_confirmados['FECHA_ACTUALIZACION'] = df_confirmados['FECHA_ACTUALIZACION'].astype(str)

#  dos formatos conocidos
opcion_iso = pd.to_datetime(df_confirmados['FECHA_ACTUALIZACION'], format='%Y-%m-%d', errors='coerce')
opcion_latina = pd.to_datetime(df_confirmados['FECHA_ACTUALIZACION'], format='%d/%m/%Y', errors='coerce')

#  Si la primera falló (es NaT), usamos la segunda
df_confirmados['FECHA_ACTUALIZACION'] = opcion_iso.fillna(opcion_latina)

nulos_finales = df_confirmados['FECHA_ACTUALIZACION'].isna().sum()


print(f"Nulos restantes: {nulos_finales}")

if nulos_finales == 0:
    print("Ya no hay nulos. Todos los formatos fueron reconocidos.")
    
    df_confirmados = df_confirmados.sort_values(by='FECHA_ACTUALIZACION').reset_index(drop=True)
    
    print(f"Rango de datos: {df_confirmados['FECHA_ACTUALIZACION'].min()} a {df_confirmados['FECHA_ACTUALIZACION'].max()}")
else:
    print(f"Todavía quedan {nulos_finales} nulos. Vamos a ver un ejemplo de qué texto tienen:")
    print(df_confirmados[df_confirmados['FECHA_ACTUALIZACION'].isna()]['FECHA_ACTUALIZACION'].unique()[:5])

Nulos restantes: 0
Ya no hay nulos. Todos los formatos fueron reconocidos.
Rango de datos: 2020-11-23 00:00:00 a 2026-01-20 00:00:00


In [53]:
df_confirmados

,FECHA_ACTUALIZACION,ID_REGISTRO,EDAD_ANOS,EDAD_MESES,EDAD_DIAS,SEXO,HABLA_LENGUA_INDIG,INDIGENA,ENTIDAD_UM_NOTIF,MUNICIPIO_UM_NOTIF,...,INSTITUCION_NOTIF,VACUNACION,EXANTEMA,FIEBRE,COMPLICACIONES,DEFUNCION,DIAGNOSTICO,CRITERIO_DIAGNOSTICO,FECHA_DIAGNOSTICO,ORIGEN_CASO
0,2020-11-23,18397,8,5,2,1,2,2,9,14,...,4,1,1,1.0,2,2,1,2,2020-03-03,4.0
1,2020-11-23,19298,38,1,2,2,2,2,9,5,...,12,2,1,1.0,2,2,1,1,2020-04-04,4.0
2,2020-11-23,19304,2,9,21,2,2,2,9,16,...,12,2,1,1.0,2,2,1,2,2020-03-31,4.0
3,2020-11-23,19307,1,11,0,2,2,2,9,16,...,12,2,1,1.0,2,2,1,2,2020-03-31,4.0
4,2020-11-23,19314,27,4,14,1,2,2,9,16,...,12,2,1,1.0,2,2,1,2,2020-03-31,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109453,2026-01-20,47664,37,4,21,2,2,2,7,78,...,5,2,1,1.0,2,2,1,2,2026-01-16,4.0
109454,2026-01-20,47663,13,4,18,2,2,2,14,70,...,12,2,1,1.0,2,2,1,2,2026-01-14,4.0
109455,2026-01-20,47662,0,4,23,2,1,1,6,4,...,20,2,1,1.0,2,2,1,2,2026-01-17,4.0
109456,2026-01-20,47679,34,0,24,2,1,1,7,78,...,5,2,1,1.0,2,2,1,2,2026-01-15,4.0
